Given input , return the balanced equation.
Eample input string: ” K Mn O 4 + H 2 O 2 = Mn O 2 + H 2 O + O 2 + K O H ”. Assume input
has spaces separating every element name and numbers as shown, also for ‘+’ and ‘=’.
Example output: ” 2 K Mn O 4 + 3 H 2 O 2 = 2 Mn O 2 + 2 H 2 O + 3 O 2 + 2 K O H ” and ” [
-2,- 3, 2, 3, 3, 2 ]”, i.e. output string for balanced chemical equation and the vector of stochiometric
coefficients with negative values for reactants.
HINT: (a) For the above reaction, atomList is ’K’, ’Mn’, ’O’, ’H’
(b) chemVectors for ‘ K Mn O 4’ is [1,1,4,0] and ’ K O H ’ is [1,0,1,1]
(c) construct matrix equation for chemical equation.
1

In [100]:
import numpy as np

In [101]:
# equation = "K Mn O 4 + H 2 O 2 = Mn O 2 + O 2 + K O H + H 2 O"
# equation = "Al + Fe 2 O 3 = Al 2 O 3 + Fe"
# equation = "H 2 + O 2 = H 2 O"
# equation = "H 2 + Cl 2 = H Cl"
# equation = " P 4 + Na O H + H 2 O = Na H 2 P O 2 + P H 3"
equation = "Cu + H N O 3 = Cu N 2 O 6 + N O + H 2 O"

In [102]:
import re
import numpy as np
import scipy.linalg as la

def findUniqueElements(equation):
    return np.unique(elementName.findall(equation))

elementPattern = re.compile(r'[A-Z]{1}[a-z]{0,}[0-9]*')
elementName = re.compile(r'[A-Z]{1}[a-z]{0,}')
elementNumber = re.compile(r'[0-9]{1,}')

def getColumnVector(element, elementVectors): #returns vector containing number of atoms of each element in compound
    elementVector = np.zeros(len(elementVectors))
    elements = elementPattern.findall(element)
    for element in elements:
        # print(element)
        name = elementName.match(element).group()
        number = elementNumber.search(element)
        if number:
            number = int(number.group())
        else:
            number = 1
        elementVector[np.where(elementVectors == name)[0][0]] = number
    return elementVector

elementVectors = findUniqueElements(equation)
print(elementVectors)

def getEquationMatrix(equation, elementVectors):
    equation = equation.replace(' ', '')
    reactants, products = equation.split('=')
    reactants = reactants.split('+')
    products = products.split('+')

    reactantMatrix = np.zeros((len(elementVectors), len(reactants)), dtype=int)
    productMatrix = np.zeros((len(elementVectors), len(products)), dtype=int)

    for i, reactant in enumerate(reactants):
        reactantMatrix[:, i] = getColumnVector(reactant, elementVectors)
    for i, product in enumerate(products):
        productMatrix[:, i] = getColumnVector(product, elementVectors)
    
    productMatrix *= -1  # Multiply product matrix by -1

    # Join reactant and product matrices
    equationMatrix = np.hstack((reactantMatrix, productMatrix))

    return equationMatrix

equationMatrix = getEquationMatrix(equation, elementVectors)
print(equationMatrix)

['Cu' 'H' 'N' 'O']
[[ 1  0 -1  0  0]
 [ 0  1  0  0 -2]
 [ 0  1 -2 -1  0]
 [ 0  3 -6 -1 -1]]


In [103]:
# solve the system
# get it to RREF 
# get the solution

nullspace = la.null_space(equationMatrix)
print(nullspace)

# get the smallest integer solution
solution = nullspace / nullspace.min()

print(solution)

[[0.29704426]
 [0.79211803]
 [0.29704426]
 [0.19802951]
 [0.39605902]]
[[1.5]
 [4. ]
 [1.5]
 [1. ]
 [2. ]]


In [104]:
# apply the coefficients to the equation

def applyCoefficients(equation, solution):
    equation = equation.replace(' ', '')
    reactants, products = equation.split('=')
    reactants = reactants.split('+')
    products = products.split('+')

    reactants = [f"{i[0]} {reactant}" for i, reactant in zip(solution[:len(reactants)], reactants)]
    products = [f"{i[0]} {product}" for i, product in zip(solution[len(reactants):], products)]

    return ' + '.join(reactants) + ' = ' + ' + '.join(products)

print(applyCoefficients(equation, solution))

2 Cu + 4 HNO3 = 2 CuN2O6 + 1 NO + 2 H2O
